In [ ]:
from google.colab  import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 6.6 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**downoad dataset** [disini](https://drive.google.com/file/d/1zqYzM11nEVU2VK81wu3uw5D5xye-OVnL/view?usp=share_link)

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/Project Akhir MSIB/data/data_news.csv')
data.head()

,judul,teks,link-href,label
0,Penyelenggaraan Pemerintahan yang Tidak Transp...,Penyelenggaraan pemerintahan yang tidak transp...,https://www.detik.com/edu/detikpedia/d-6373276...,1
1,"Sosialisasi Politik: Definisi, Cara Kerja, dan...",Sosialisasi politik tentunya sangat penting da...,https://www.detik.com/edu/detikpedia/d-6338406...,1
2,"Kekuasaan Kongsi Dagang VOC di Indonesia, Puny...",Kekuasaan Kongsi Dagang VOC di Indonesia Punya...,https://www.detik.com/edu/detikpedia/d-6329307...,1
3,Mahfud Posting Foto Hitam Putih: Pemerintah Se...,(Menko Polhukam) Mahfud Md menyampaikan sikap ...,https://news.detik.com/berita/d-6323915/mahfud...,1
4,Luhut Minta Pemerintahan-BUMN Tak Pakai Produk...,Menko Marves Luhut Binsar Pandjaitan meminta k...,https://finance.detik.com/detiktv/d-6319824/lu...,1


In [ ]:
from nltk import corpus
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('stopwords')
factory = StemmerFactory()
stemmer = factory.create_stemmer()

corpus = []

for i in range(0, len(data)):
    text = re.sub('[^a-zA-Z]', ' ', str(data['teks'][i]))
    text = text.lower()
    text = text.split()
    text = [stemmer.stem(word) for word in text if not word in stopwords.words('indonesian')]
    text = ' '.join(text)
    corpus.append(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))

X = tfidf_v.fit_transform(corpus).toarray()
y = data['label']

In [ ]:
# Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Metode Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)

GaussianNB()

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_NB = classifierNB.predict(X_test) 
cm_NB = confusion_matrix(y_test, y_pred_NB)

print("Confusion Matrix",cm_NB)

Confusion Matrix [[173  23]
 [ 10 195]]


In [ ]:
akurasi_NB = ((cm_NB[0][0]+cm_NB[1][1])/(cm_NB[0][0]+cm_NB[1][1]+cm_NB[0][1]+cm_NB[1][0]))*100
print("Akurasi",akurasi_NB)

Akurasi 91.77057356608479


In [ ]:
import pickle
pickle.dump(classifierNB, open('model.pkl', 'wb'))
pickle.dump(tfidf_v, open('tfidfvect.pkl', 'wb'))

In [ ]:
# Load model and vectorizer
joblib_model = pickle.load(open('model.pkl', 'rb'))
joblib_vect = pickle.load(open('tfidfvect.pkl', 'rb'))
val_pkl = joblib_vect.transform([text]).toarray()
joblib_model.predict(val_pkl)

array([1])